In [1]:
# Set up logging

import logging

logging.basicConfig(
    format="[%(asctime)s %(levelname)s %(threadName)s %(name)s:%(funcName)s:%(lineno)s] %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S%z",
)
logging.getLogger("council").setLevel(logging.DEBUG)

In [2]:
# Council imports

from council.runners import Budget
from council.contexts import AgentContext, ChatHistory
from council.agents import Agent
from council.chains import Chain
from council.llm.openai_llm_configuration import OpenAILLMConfiguration
from council.llm.openai_llm import OpenAILLM

from skills import SectionWriterSkill, OutlineWriterSkill
from controller import WritingAssistantController
from evaluator import BasicEvaluatorWithSource

In [3]:
# Read environment variables and initialize OpenAI LLM instance

import dotenv
dotenv.load_dotenv()
openai_llm = OpenAILLM(config=OpenAILLMConfiguration.from_env())

In [4]:
# Create Skills

outline_skill = OutlineWriterSkill(openai_llm)
writing_skill = SectionWriterSkill(openai_llm)

In [5]:
# Create Chains

outline_chain = Chain(
    name="Outline Writer",
    description="Write or revise the outline (i.e. section headers) of a research article in markdown format. Only use this chain when the ARTICLE OUTLINE is missing or incomplete.",
    runners=[outline_skill]
)

writer_chain = Chain(
    name="Article Writer",
    description="Write or revise specific section bodies of a research article in markdown format. Use this chain to write the main research article content.",
    runners=[writing_skill]
)

In [6]:
# Create Controller

controller = WritingAssistantController(
    openai_llm,
    top_k_execution_plan=5
)

In [7]:
# Initialize Agent

chat_history = ChatHistory()
run_context = AgentContext(chat_history)
agent = Agent(controller, [outline_chain, writer_chain], BasicEvaluatorWithSource())

In [8]:
# Run Agent

chat_history.add_user_message("Write a detailed research article about the history of box manufacturing.")
result = agent.execute(run_context, Budget(1800))

[2023-07-27 12:37:19-0400 INFO MainThread council.agents.agent:execute:60] message="agent execution started"
[2023-07-27 12:37:19-0400 INFO MainThread council.agents.agent:execute:62] message="agent iteration started" iteration="1"
[2023-07-27 12:37:19-0400 DEBUG MainThread council.llm.llm_base:post_chat_request:30] message="starting execution of llm request"
[2023-07-27 12:37:19-0400 DEBUG MainThread council.llm.openai_chat_completions_llm:_post_request:145] message="Sending chat GPT completions request" payload="{'temperature': 0.1, 'n': 1, 'model': 'gpt-4', 'messages': [{'role': 'system', 'content': 'You are the Controller module for an AI assistant built to write and revise research articles.'}, {'role': 'user', 'content': "\n        # Task Description\n        Your task is to decide how best to write or revise the ARTICLE. Considering the ARTICLE OUTLINE, ARTICLE, and the CONVERSATION HISTORY,\n        use your avaiable CHAINS to decide what steps to take next. You are not respons

In [11]:
print(result.messages[-1].message.message)

# Introduction

The history of box manufacturing is a captivating exploration that traces the evolution of a simple yet indispensable item in our daily lives. The box, in its various forms and materials, has been a fundamental component of commerce, storage, and transportation for centuries. This article aims to delve into the rich history of box manufacturing, exploring its origins, development, and the technological advancements that have shaped it over time.

The significance of box manufacturing cannot be overstated. Boxes are ubiquitous, serving a myriad of purposes in different sectors. From packaging consumer goods to storing valuable items, from facilitating global trade to aiding in logistics and supply chain management, boxes play a crucial role. They are an integral part of our economy and society, and their production and use have significant environmental implications.

Understanding the history of box manufacturing provides valuable insights into the evolution of industri

In [13]:
chat_history.add_user_message("Can you please add a section (including subsections) on boxes in popular culture?")
result = agent.execute(run_context, Budget(600))

[2023-07-27 13:13:49-0400 INFO MainThread council.agents.agent:execute:60] message="agent execution started"
[2023-07-27 13:13:49-0400 INFO MainThread council.agents.agent:execute:62] message="agent iteration started" iteration="4"
[2023-07-27 13:13:49-0400 DEBUG MainThread council.llm.llm_base:post_chat_request:30] message="starting execution of llm request"
[2023-07-27 13:13:49-0400 DEBUG MainThread council.llm.openai_chat_completions_llm:_post_request:145] message="Sending chat GPT completions request" payload="{'temperature': 0.1, 'n': 1, 'model': 'gpt-4', 'messages': [{'role': 'system', 'content': 'You are the Controller module for an AI assistant built to write and revise research articles.'}, {'role': 'user', 'content': "\n        # Task Description\n        Your task is to decide how best to write or revise the ARTICLE. Considering the ARTICLE OUTLINE, ARTICLE, and the CONVERSATION HISTORY,\n        use your avaiable CHAINS to decide what steps to take next. You are not respons

In [15]:
print(result.messages[-1].message.message)

# Introduction

The history of box manufacturing is a captivating exploration that traces the evolution of a simple yet indispensable item in our daily lives. The box, in its various forms and materials, has been a fundamental component of commerce, storage, and transportation for centuries. This article aims to delve into the rich history of box manufacturing, exploring its origins, development, and the technological advancements that have shaped it over time.

The significance of box manufacturing cannot be overstated. Boxes are ubiquitous, serving a myriad of purposes in different sectors. From packaging consumer goods to storing valuable items, from facilitating global trade to aiding in logistics and supply chain management, boxes play a crucial role. They are an integral part of our economy and society, and their production and use have significant environmental implications.

Understanding the history of box manufacturing provides valuable insights into the evolution of industri